Fairville ist eine Kleinstadt mit zirka 20,000 Einwohnern. Die jährliche Steuerbemessungsgrundlage
für Grundbesitz beträgt 550 Mio. USD. Die jährlichen Steuerbemessungsgrundlagen für Lebensmittel und Medikamente sowie für den allgemeinen Verkauf belaufen sich auf 35 Mio. USD bzw. 55 Mio. USD. Der jährliche lokale Benzinverbrauch
wird auf 7.5 Millionen Gallonen geschätzt. Der Stadtrat möchte die Steuersätze auf der Grundlage von vier
Hauptzielen entwickeln:

1. Die Steuereinnahmen müssen mindestens 16 Millionen Dollar betragen, um die finanziellen Verpflichtungen der Stadt zu erfüllen.
2. Die Steuern auf Lebensmittel und Arzneimittel dürfen 10 % aller eingenommenen Steuern nicht übersteigen.
3. Die allgemeinen Verkaufssteuern dürfen 20 % aller eingenommenen Steuern nicht überschreiten.
4. Die Benzinsteuer darf 2 Cent pro Gallone nicht überschreiten.


In [42]:
%pip install --quiet gurobipy
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

Note: you may need to restart the kernel to use updated packages.


In [43]:
minf = gp.Model("minf")

# Create variables

x = minf.addVar(vtype=GRB.CONTINUOUS, name="x")
y = minf.addVar(vtype=GRB.CONTINUOUS, name="y")

# Set objective function
minf.setObjective(x + y, GRB.MINIMIZE)

# Add constraints
minf.addConstr(x + y >= 20, "c0")
minf.addConstr(2 * x - y <= 1, "c1")
minf.addConstr(x <= 2, "c2")
minf.addConstr(y <= 5, "c3")
minf.addConstr(x >= 1, "c4")
minf.addConstr(y >= 0.5, "c5")
minf.addConstr(-x + y <= 10, "c6")

# Optimize model
minf.optimize()

# Print the model status

if minf.status == GRB.OPTIMAL:
    print("Optimal solution found")
    print(f"x: {x.X}")
    print(f"y: {y.X}")
    print(f"Objective value: {minf.ObjVal}")
else:
    print("No optimal solution found")
    if minf.status == GRB.INFEASIBLE:
        print("Model is infeasible")
    elif minf.status == GRB.UNBOUNDED:
        print("Model is unbounded")
    else:
        print(f"Model status: {minf.status}")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 7 rows, 2 columns and 10 nonzeros
Model fingerprint: 0x9a3b66b8
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 2e+01]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model
No optimal solution found
Model is infeasible


In [44]:
minf.computeIIS()
minf.write("model.ilp")

with open("model.ilp", "r") as f:
    lines = f.readlines()
    for line in lines:
        print(line.strip())


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.150000e+01   0.000000e+00      0s

IIS computed: 3 constraints and 0 bounds
IIS runtime: 0.00 seconds (0.00 work units)
\ Model minf_copy
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize

Subject To
c0: x + y >= 20
c2: x <= 2
c3: y <= 5
Bounds
x free
y free
End


In [45]:
minf.feasRelaxS(relaxobjtype=0, minrelax=False, vrelax=False, crelax=1)
minf.optimize()

minf.write("model_relaxed.lp")

with open("model_relaxed.lp", "r") as f:
    print(f.read())

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 7 rows, 9 columns and 17 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 2e+01]
LP warm-start: use basis

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.900000e+01   0.000000e+00      0s
       2    1.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.300000000e

In [14]:
# Create a new model

m = gp.Model("Tax planning")

# Create variables

x_re = m.addVar(name="RE")
x_fad = m.addVar(name="FAD")
x_sales = m.addVar(name="Sales")
x_gas = m.addVar(name="Gasoline")

# Einnahmen

revenue = 550 * x_re + 35 * x_fad + 55 * x_sales + 0.75 * x_gas

# Add constraints

m.addConstr(revenue >= 16, "Min Revenue")
m.addConstr(35 * x_fad <= 0.001 * revenue, "Food and drugs constraint")
m.addConstr(55 * x_sales <= 0.002 * revenue, "Sales constraint")
m.addConstr(x_gas <= 0.01, "Gasoline constraint")

# Zielfunktion

m.setObjective(x_fad, GRB.MAXIMIZE)

m.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 4 rows, 4 columns and 13 nonzeros
Model fingerprint: 0xe141fcfe
Coefficient statistics:
  Matrix range     [8e-04, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 2e+01]
Presolve removed 3 rows and 1 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


In [8]:
GRB.INFEASIBLE

3

In [13]:
m.Status

2

In [10]:
%pip install --quiet gurobipy
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Create a new model

m1 = gp.Model("Tax planning")

# Create variables

x_re = m1.addVar(name="RE")
x_fad = m1.addVar(name="FAD")
x_sales = m1.addVar(name="Sales")
x_gas = m1.addVar(name="Gasoline")

revenue = 550 * x_re + 35 * x_fad + 55 * x_sales + 0.75 * x_gas

# Add constraints

rev_p = m1.addVar(name="revenue_plus")
rev_n = m1.addVar(name="revenue_minus")

fad_p = m1.addVar(name="food_plus")
fad_n = m1.addVar(name="food_minus")


sales_p = m1.addVar(name="sales_plus")
sales_n = m1.addVar(name="sales_minus")

gas_p = m1.addVar(name="gas_plus")
gas_n = m1.addVar(name="gas_minus")

m1.addConstr(revenue + rev_p - rev_n == 16, "Min Revenue")
m1.addConstr(35 * x_fad - 0.1 * revenue + fad_p - fad_n == 0, "Food and drugs constraint")
m1.addConstr(55 * x_sales - 0.2 * revenue + sales_p - sales_n == 0, "Sales constraint")
m1.addConstr(x_gas + gas_p - gas_n == 2, "Gasoline constraint")

m1.setObjective(rev_n + fad_n + sales_n + gas_n, GRB.MINIMIZE)

m1.optimize()


Note: you may need to restart the kernel to use updated packages.
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 4 rows, 12 columns and 21 nonzeros
Model fingerprint: 0x699e1766
Coefficient statistics:
  Matrix range     [8e-02, 6e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 2e+01]
Presolve removed 4 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [11]:
%pip install --quiet gurobipy
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

# Create a new model

m2 = gp.Model("Tax planning")

# Create variables

x_re = m2.addVar(name="RE")
x_fad = m2.addVar(name="FAD")
x_sales = m2.addVar(name="Sales")
x_gas = m2.addVar(name="Gasoline")

revenue = 550 * x_re + 35 * x_fad + 55 * x_sales + 0.75 * x_gas

m2.setObjectiveN(revenue - 16, index=0, weight=1, name="Revenue")
m2.setObjectiveN(35 * x_fad - 0.1 * revenue, index=1, weight=1, name="FAD")
m2.setObjectiveN(55 * x_sales - 0.2 * revenue, index=2, weight=1, name="Sales")
m2.setObjectiveN(x_gas - 2, index=3, weight=-1, name="Gasoline")

m2.write("tax_planning.lp")

with open("tax_planning.lp", "r") as file:
    content = file.read()
    print(content)

m2.optimize()


Note: you may need to restart the kernel to use updated packages.
\ Model Tax planning
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize multi-objectives
  Revenue: Priority=0 Weight=1 AbsTol=1e-06 RelTol=0
   550 RE + 35 FAD + 55 Sales + 0.75 Gasoline - 16 Constant0
  FAD: Priority=0 Weight=1 AbsTol=1e-06 RelTol=0
   - 55 RE + 31.5 FAD - 5.5 Sales - 0.075 Gasoline
  Sales: Priority=0 Weight=1 AbsTol=1e-06 RelTol=0
   - 110 RE - 7 FAD + 44 Sales - 0.15 Gasoline
  Gasoline: Priority=0 Weight=-1 AbsTol=1e-06 RelTol=0
   Gasoline - 2 Constant3
Subject To
Bounds
 Constant0 = 1
 Constant3 = 1
End

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125U, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 0 rows, 4 columns and 0 nonzeros
Model fingerprint: 0x8c6be8c3
Variable types: 4 continuous, 0 integer